##Library Imports

In [49]:
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF

##Double Convolutions

In [50]:
class DoubleConv2D(nn.Module):
  def __init__(self, in_channels, out_channels):
    super().__init__()
    self.Conv =nn.Sequential(
    nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias = False),
    nn.BatchNorm2d(out_channels),
    nn.ReLU(inplace = True),
    nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias = False),
    nn.BatchNorm2d(out_channels),
    nn.ReLU(inplace = True)
    )

  def forward(self, x):
    return self.Conv(x)

##UNET

In [51]:
class UNET(nn.Module):
  def __init__(self,in_channels=3 ,out_channels=1 ,features = [64,128,256,512] ):
    super().__init__()
    self.ups = nn.ModuleList()
    self.downs = nn.ModuleList()
    self.pool = nn.MaxPool2d(kernel_size=2, stride= 2)

    ##Down sampling of the layers
    for feature in features:
      self.downs.append(DoubleConv2D(in_channels,feature))
      in_channels = feature
    ##Up sampling of the layers
    for feature in reversed(features):
      self.ups.append(nn.ConvTranspose2d(feature*2,feature,2,2))
      self.ups.append(DoubleConv2D(feature*2,feature))

    self.bottleneck = DoubleConv2D(features[-1],features[-1]*2)
    self.final_conv = nn.Conv2d(features[0],out_channels,kernel_size = 1)
  ##Forward
  def forward(self,x):
    skip_connections = []

    for down in self.downs:
      x = down(x)
      skip_connections.append(x)
      x = self.pool(x)

    x = self.bottleneck(x)
    skip_connections = skip_connections[::-1]

    for idx in range(0,len(self.ups),2):
      x = self.ups[idx](x)
      skip_connection = skip_connections[idx//2]
      if x.shape != skip_connection.shape:
        x = TF.resize(x, size = skip_connection.shape[2:] )

      concat_skip = torch.cat((skip_connection,x),dim = 1)
      x = self.ups[idx +1](concat_skip)

    return self.final_conv(x)


##Test

In [54]:
def Test():
  x = torch.randn((3,1,161,161))
  model = UNET(in_channels=1,out_channels = 1)
  preds = model(x)
  print(x.shape)
  assert preds.shape == x.shape

##Execution

In [55]:
if __name__ == "__main__":
  Test()

torch.Size([3, 1, 161, 161])
